In [2]:
import itertools
from collections import Counter
from parse import load_dataframes
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import folium

In [3]:
import pandas as pd
df = pd.DataFrame(data=np.array([[1, 2, 3], [1, 5, 6], [7, 8, 9]]), columns=['A', 'B', 'C'])
display(df)

,A,B,C
0,1,2,3
1,1,5,6
2,7,8,9


In [2]:
def set_config():
    # 폰트, 그래프 색상 설정
    font_list = fm.findSystemFonts(fontpaths=None, fontext="ttf")
    if any(["notosanscjk" in font.lower() for font in font_list]):
        plt.rcParams["font.family"] = "Noto Sans CJK JP"
    else:
        if not any(["malgun" in font.lower() for font in font_list]):
            raise Exception(
                "Font missing, please install Noto Sans CJK or Malgun Gothic. If you're using ubuntu, try `sudo apt install fonts-noto-cjk`"
            )

        plt.rcParams["font.family"] = "Malgun Gothic"

    sns.set_palette(sns.color_palette("Spectral"))
    plt.rc("xtick", labelsize=6)

In [7]:
df.drop(0)

,A,B,C
1,1,5,6
2,7,8,9


In [19]:
dataframes = load_dataframes()
min_reviews = 10
stores_reviews = pd.merge(dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store")
scores_group = stores_reviews.groupby(["store", "store_name"])
scores = scores_group.agg(count=('store', 'size'), score=('score', 'mean'))
scores = scores[scores["count"]>min_reviews]
stores = pd.merge(scores, dataframes["stores"], left_on="store", right_on="id")
stores

,count,score,id,store_name,branch,area,tel,address,latitude,longitude,category
0,11,3.272727,149,101번지 남산돈까스,None,남산,02-777-7929,서울특별시 중구 남산동2가 49-24,37.556959,126.985722,왕돈까스|돈까스
1,12,5.000000,221,10Q,,송파나루역,010-9590-8525,서울특별시 송파구 송파동 131-1,37.508087,127.112586,이자카야|유린기
2,11,5.000000,2476,4SEED,None,평창동,02-395-7187,서울특별시 종로구 평창동 273-1,37.604625,126.965985,오가닉|레스토랑
3,16,4.125000,2492,4브라더스,None,도곡동,02-574-4147,서울특별시 강남구 도곡동 423-5,37.484028,127.045280,월남쌈|나시고랭
4,17,3.882353,3271,60년전통 할매국밥,None,범일동,051-646-6295,부산광역시 동구 범일동 28-5,35.141011,129.056167,돼지국밥|수육백반
...,...,...,...,...,...,...,...,...,...,...,...
694,14,3.285714,357248,주은감자탕,None,삼전동,02-413-3670,서울특별시 송파구 삼전동 132-14,37.503682,127.096814,감자탕
695,15,3.400000,358891,중림장,None,충정로,02-392-7743,서울특별시 중구 중림동 468,37.560264,126.967707,도가니탕|설렁탕
696,20,3.600000,359074,중앙떡볶이,None,대구,053-424-7692,대구시 중구 동성로2가 70,35.869512,128.596818,납작만두|떡볶이
697,33,4.212121,359300,중앙해장,None,삼성역,02-558-7905,서울특별시 강남구 대치동 996-16,37.508313,127.065468,곱창전골|내장탕


In [14]:
from folium.plugins import MarkerCluster
m = folium.Map(location=[36.3504567,127.3848187],zoom_start=12)
marker_cluster = MarkerCluster().add_to(m)

In [20]:
stores = stores[stores.address.str.contains("대전")==True]
stores

,count,score,id,store_name,branch,area,tel,address,latitude,longitude,category
26,18,4.000000,21082,개천식당,None,대전,042-256-5627,대전광역시 동구 원동 62-1,36.328617,127.431731,이북식만두|만두국
59,14,3.000000,35747,광천식당,None,대전,042-226-4751,대전광역시 중구 선화동 52-2,36.328571,127.423591,두부두루치기|오징어두루치기
74,11,3.727273,43425,권인순 갈비김치찌개,None,노은동,042-477-8529,대전광역시 유성구 노은동 516-8,36.367519,127.316800,갈비김치찌개|김치찌개
320,15,3.666667,164893,바로그집,None,은행동,042-254-8925,대전광역시 중구 은행동 216,36.326667,127.421874,떡볶이|라볶이
328,29,4.172414,168290,반갱,None,둔산동,042-486-0950,대전광역시 서구 둔산동 991,36.353295,127.377268,한식레스토랑|퓨전한식
428,13,2.384615,222420,설해돈,None,둔산동,042-484-1119,대전광역시 서구 둔산동 993 라온채빌딩 4층,36.353742,127.377270,매운갈비찜|돈까스
431,32,4.093750,223421,성심당,None,대전,042-256-4114,대전시 중구 은행동 145,36.327761,127.427267,튀김소보로|부추빵
432,17,4.235294,223423,성심당,대전역점,대전역,042-220-4138,대전시 동구 정동 1-1 대전역사내 2층,36.331498,127.432999,튀김소보로|부추빵
459,16,3.562500,239029,숯골원냉면,None,신성동,042-861-3287,대전시 유성구 신성동 136-2,36.390145,127.350208,평양냉면|냉면
465,13,3.846154,241109,스바라시,None,유성,042-477-7909,대전시 유성구 봉명동 611-1,36.359410,127.344480,라멘|미소라멘


In [21]:
for i in stores.index:
    name = stores.loc[i, "store_name"]
    lat = stores.loc[i, "latitude"]
    lon = stores.loc[i, "longitude"]
    if name and lat and lon:
        folium.Marker(
            location=[lat,lon],
            popup=name,
        ).add_to(marker_cluster)

In [22]:
m